In [2]:
import pandas as pd

# 엑셀 파일 불러오기
excel_data = pd.ExcelFile("직업별_하이브리드2.xlsx", engine="openpyxl")
sheet_names = excel_data.sheet_names

# 'Sheet1' 시트 불러오기
df = excel_data.parse('Sheet1')

# 상위 5개 행 확인
df.head()


,Unnamed: 0,<성격> 성취/노력,<성격> 인내,<성격> 책임성과 진취성,<성격> 리더십,<성격> 협조,<성격> 타인에대한 배려,<성격> 사회성,<성격> 자기통제,<성격> 스트레스감내성,<성격> 적응성/융통성,<성격> 신뢰성,<성격> 꼼꼼함,<성격> 정직성,<성격> 독립성,<성격> 혁신,<성격> 분석적 사고
0,3D 프린팅모델러,-0.004824,0.006580,0.021130,0.034288,-0.005045,-0.023341,0.061026,0.051478,-0.024549,-0.042125,0.043086,-0.001368,-0.028367,-0.038604,0.073231,-0.035807
1,IT 기술지원 전문가,0.012055,0.000335,-0.004611,-0.020959,-0.015140,0.003798,0.011462,-0.001486,-0.088822,0.063419,0.000248,0.000837,0.016483,-0.000738,-0.008149,0.010450
2,IT 테스터 및 IT QA 전문가,0.009948,0.032962,0.000190,-0.010775,-0.001954,0.007174,0.005111,-0.009550,-0.000838,-0.006460,-0.012090,0.000847,0.011726,0.014760,-0.069169,0.018896
3,가상(증강)현실전문가,0.042578,-0.013506,0.073926,0.061150,0.000415,-0.013782,0.022501,-0.001368,0.021412,0.010201,-0.023906,-0.034839,0.042785,-0.018650,-0.007275,-0.011197
4,감사 사무원,0.006162,0.010378,-0.015195,0.057472,-0.001732,0.002709,0.040671,0.013038,-0.066726,-0.001054,0.002250,0.011937,-0.255377,-0.024031,0.045683,0.013703


In [3]:
from sklearn.preprocessing import normalize

# 1. 성격 가중치 부분만 추출 (두 번째 열부터 끝까지)
trait_columns = df.columns[1:]
trait_matrix = df[trait_columns].values

# 2. L2 정규화 적용 (직업별로 정규화)
normalized_matrix = normalize(trait_matrix, axis=1)

# 3. 정규화된 결과를 DataFrame으로 변환
df_l2_normalized = pd.DataFrame(normalized_matrix, columns=trait_columns)
df_l2_normalized.insert(0, '직업명', df.iloc[:, 0])  # 직업명 다시 삽입


In [4]:
import pandas as pd

# 이미 존재하는 정규화된 데이터프레임 사용: df_l2_normalized
trait_columns = df_l2_normalized.columns[1:]  # 첫 번째 열은 직업명

# 사용자 성격 점수 입력 (0~2점)
def get_user_input(trait_columns):
    print("각 성격 항목에 대해 0~2 사이의 점수를 입력하세요. (엔터 시 기본값 1)")
    user_input = {}
    for trait in trait_columns:
        try:
            val = input(f"{trait}: ")
            val = int(val) if val.strip() != "" else 1
            if val < 0 or val > 2:
                print("⚠️ 0~2 사이의 값만 허용됩니다. 기본값 1이 적용됩니다.")
                val = 1
        except:
            print("⚠️ 숫자로 입력해주세요. 기본값 1이 적용됩니다.")
            val = 1
        user_input[trait] = val
    return pd.Series(user_input)

# 만족도 계산 함수
def calculate_job_satisfaction(df_normalized, user_traits):
    scores = df_normalized[trait_columns].multiply(user_traits, axis=1).sum(axis=1)
    results = pd.DataFrame({
        '직업명': df_normalized['직업명'],
        '예상직업만족도': scores
    }).sort_values(by='예상직업만족도', ascending=False).reset_index(drop=True)
    return results

# 실행
user_traits = get_user_input(trait_columns)
satisfaction_results = calculate_job_satisfaction(df_l2_normalized, user_traits)

# 상위 결과 출력
print("\n📌 예상 직업만족도 상위 추천 결과:")
print(satisfaction_results.head(10))


각 성격 항목에 대해 0~2 사이의 점수를 입력하세요. (엔터 시 기본값 1)

📌 예상 직업만족도 상위 추천 결과:
                  직업명  예상직업만족도
0           3D 프린팅모델러      0.0
1        전기·전자 공학 시험원      0.0
2             인공지능전문가      0.0
3        인사·교육·훈련 사무원      0.0
4  인적자원전문가(전직지원전문가포함)      0.0
5             일반행정공무원      0.0
6           임상시험코디네이터      0.0
7   임업 및 산림 연구원 및 기술자      0.0
8            자동차공학기술자      0.0
9            자연과학 시험원      0.0


## 